In [ ]:
### Import Script ####

# Run to Import and process data from CSV file in raw folder 
# def import_script():

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter('ignore')

In [9]:
client_train_import = pd.read_csv('data/raw/client_train.csv', low_memory=False)
invoice_train_import = pd.read_csv('data/raw/invoice_train.csv', low_memory=False)

client_test_import = pd.read_csv('data/raw/client_test.csv', low_memory=False)
invoice_test_import = pd.read_csv('data/raw/invoice_test.csv', low_memory=False)


In [10]:
client_train = client_train_import.copy()
invoice_train = invoice_train_import.copy()

client_test = client_test_import.copy() 
invoice_test = invoice_test_import.copy()

In [11]:
#changing dates to datetime format
for df in [invoice_train,invoice_test]:
    df['invoice_date'] = pd.to_datetime(df['invoice_date'])

In [12]:
counter_info_train = invoice_train['counter_type']
counter_info_test = invoice_test['counter_type']

invoice_train = pd.get_dummies(invoice_train, columns=['counter_type'], dtype=int)
invoice_test = pd.get_dummies(invoice_test, columns=['counter_type'], dtype=int)

invoice_train = pd.concat([invoice_train, counter_info_train], axis=1)
invoice_test = pd.concat([invoice_test, counter_info_test], axis=1)  

d={"ELEC":0,"GAZ":1}
invoice_train['counter_type']=invoice_train['counter_type'].map(d)
invoice_test['counter_type']=invoice_test['counter_type'].map(d)


In [13]:
invoice_train.head()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type_ELEC,counter_type_GAZ,counter_type
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,0,0,14302,14384,4,1,0,0
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,0,0,12294,13678,4,1,0,0
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,0,0,14624,14747,4,1,0,0
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,0,0,14747,14849,4,1,0,0
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,0,0,15066,15638,12,1,0,0


In [14]:
client_train['client_catg'] = client_train['client_catg'].astype(int)
client_train['disrict'] = client_train['disrict'].astype(int)

client_test['client_catg'] = client_test['client_catg'].astype(int)
client_test['disrict'] = client_test['disrict'].astype(int)

# def aggregate_by_client_id(invoice_data):
#     aggs = {}
#     aggs['consommation_level_1'] = ['mean']
#     aggs['consommation_level_2'] = ['mean']
#     aggs['consommation_level_3'] = ['mean']
#     aggs['consommation_level_4'] = ['mean']
#     aggs['counter_type_ELEC'] = ['mean']
#     aggs['counter_type_GAZ'] = ['mean']

#     agg_trans = invoice_data.groupby(['client_id']).agg(aggs)
#     agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
#     agg_trans.reset_index(inplace=True)

#     df = (invoice_data.groupby('client_id')
#             .size()
#             .reset_index(name='{}transactions_count'.format('1')))
    
#     dfreturn = pd.merge(df, agg_trans, on='client_id', how='left')

#     return dfreturn

In [15]:
def aggregate_by_client_id(invoice_data):
    aggs = {
        'consommation_level_1': ['mean'],
        'consommation_level_2': ['mean'],
        'consommation_level_3': ['mean'],
        'consommation_level_4': ['mean'],
        'counter_type_ELEC': ['mean'],
        'counter_type_GAZ': ['mean'],
        'counter_type': ['mean']            
    }

    agg_trans = invoice_data.groupby(['client_id']).agg(aggs)
    agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)

    df = (invoice_data.groupby(['client_id'])
            .size()
            .reset_index(name='transactions_count'))

    # Count transactions grouped by counter_type
    counter_type_counts = invoice_data.groupby(['client_id', 'counter_type']).size().unstack(fill_value=0)
    counter_type_counts.columns = [f'transactions_count_{col}' for col in counter_type_counts.columns]
    counter_type_counts.reset_index(inplace=True)

    dfreturn = pd.merge(df, agg_trans, on='client_id', how='left')
    dfreturn = pd.merge(dfreturn, counter_type_counts, on='client_id', how='left')

    return dfreturn

In [23]:
agg_train = aggregate_by_client_id(invoice_train)

train = pd.merge(client_train,agg_train, on='client_id', how='left')

agg_test = aggregate_by_client_id(invoice_test)
test = pd.merge(client_test,agg_test, on='client_id', how='left')

drop_columns = ['client_id', 'creation_date']

In [24]:
train.sort_values(by='counter_type_ELEC_mean', ascending=True).head(10)

,disrict,client_id,client_catg,region,creation_date,target,transactions_count,consommation_level_1_mean,consommation_level_2_mean,consommation_level_3_mean,consommation_level_4_mean,counter_type_ELEC_mean,counter_type_GAZ_mean,counter_type_mean,transactions_count_0,transactions_count_1
56673,60,train_Client_2906,11,101,01/11/1995,0.0,3,484.000000,0.0,0.0,0.0,0.0,1.0,1.0,0,3
793,60,train_Client_10071,11,101,18/09/2009,0.0,21,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0,21
133563,60,train_Client_98261,11,101,01/02/2005,0.0,39,113.717949,0.0,0.0,0.0,0.0,1.0,1.0,0,39
55436,60,train_Client_27947,11,101,28/04/2009,0.0,2,189.000000,0.0,0.0,0.0,0.0,1.0,1.0,0,2
123305,69,train_Client_89029,12,104,10/04/2008,0.0,17,809.411765,0.0,0.0,0.0,0.0,1.0,1.0,0,17
79146,60,train_Client_49286,11,101,28/06/2007,0.0,14,1.285714,0.0,0.0,0.0,0.0,1.0,1.0,0,14
105443,69,train_Client_72953,11,104,27/05/1993,0.0,1,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0,1
92444,60,train_Client_61253,11,101,31/10/2009,0.0,1,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0,1
105486,60,train_Client_72992,51,101,16/11/2013,0.0,42,1611.595238,0.0,0.0,0.0,0.0,1.0,1.0,0,42
55213,62,train_Client_27746,11,301,07/04/2015,0.0,9,308.111111,0.0,0.0,0.0,0.0,1.0,1.0,0,9


In [ ]:
drop_columns = ['client_id', 'counter_type_ELEC_mean', 'counter_type_GAZ_mean', 'counter_type_mean']

for col in drop_columns:
    if col in train.columns:
        train.drop([col], axis=1, inplace=True)
    if col in test.columns:
        test.drop([col], axis=1, inplace=True)


In [27]:
train.head()

,disrict,client_catg,region,creation_date,target,transactions_count,consommation_level_1_mean,consommation_level_2_mean,consommation_level_3_mean,consommation_level_4_mean,transactions_count_0,transactions_count_1
0,60,11,101,31/12/1994,0.0,35,352.400000,10.571429,0.000000,0.000000,35,0
1,69,11,107,29/05/2002,0.0,37,557.540541,0.000000,0.000000,0.000000,37,0
2,62,11,301,13/03/1986,0.0,18,798.611111,37.888889,0.000000,0.000000,18,0
3,69,11,105,11/07/1996,0.0,20,1.200000,0.000000,0.000000,0.000000,20,0
4,62,11,303,14/10/2014,0.0,14,663.714286,104.857143,117.357143,36.714286,14,0


In [ ]:
train.to_csv('data/data_processed/client_data.csv', index=False)
test.to_csv('data/data_processed/validation.csv', index=False)